In [ ]:
# lo_polygon = ox.geocode_to_gdf('R176095', by_osmid=True).to_crs(epsg=32636)
# spb_polygon = ox.geocode_to_gdf('R337422', by_osmid=True).to_crs(epsg=32636).buffer(3000)
# city = lo_polygon.union(spb_polygon).to_crs(epsg=4326) #  get lo polygon
# polygon = gpd.GeoDataFrame([{'name': 'Ленинградская область', 'geometry': city[0]}], crs="EPSG:4326")

In [ ]:
# gg= he.convert_list_attr_to_str(g.graph)
# for node,e,data in gg.edges(data=True):
#     if 'geometry' in data:
#         data['geometry']=str(data['geometry'])
# nx.write_graphml(gg, f"tula.graphml")

In [ ]:
import sys
sys.path.append('/Users/polina/Desktop/github/transport_frames')
from transport_frames.graphbuilder.graph import Graph 
from transport_frames.frame_grader.advanced_grade import AdvancedGrader
from transport_frames.framebuilder.frame import Frame
import transport_frames.utils.helper_funcs as he
from transport_frames.indicators.indicator_terr import indicator_territory
from transport_frames.indicators.indicator_area import indicator_area
from transport_frames.indicators.utils import assign_services_names_to_nodes, availability_matrix
import osmnx as ox
import geonetworkx as gnx
import geopandas as gpd
import networkx as nx
from shapely import Polygon, MultiPolygon
import momepy
import shapely
from dongraphio import GraphType


## create graph and frame

In [ ]:
# polygon regions cities
regions = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/regions_of_russia.geojson')
regions = regions[regions['ISO3166-2']!='RU-CHU']
regions = regions[regions['geometry'].apply(lambda x: isinstance(x, (Polygon, MultiPolygon)))]

polygon = ox.geocode_to_gdf('R81993',by_osmid=True)

cities = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Тульская_область_region_points.geojson')
cities = cities[cities.is_admin_centre_district==1][['name','geometry']].reset_index()

In [ ]:
local_crs = 32637

In [ ]:
# g = Graph.from_polygon(polygon, crs=local_crs)

tula = he.convert_geometry_from_wkt(nx.read_graphml('/Users/polina/Desktop/github/transport_frames/transport_frames/tula.graphml'))
tula = he.convert_list_attr_from_str(tula)
g =  Graph(tula,crs=local_crs)

In [ ]:
f = Frame(g.graph, regions, polygon, cities)

## grade terr (by frame)

In [ ]:
poly = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/settlement.geojson')

In [ ]:
graded_terr = f.grade_territory(poly, include_priority=True)

## grade terr (advanced)

In [ ]:
points = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Тульская_область_region_points.geojson')
settlements = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/settlement.geojson')
inter = he.prepare_graph(nx.read_graphml('/Users/polina/Desktop/github/transport_frames/preprocessed/Тульская_область/Тульская_область_inter.graphml'))
r_stops = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/railway_station.geojson')
b_stops = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/bus_stop.geojson')
aero = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/aerodrome.geojson')

In [ ]:
adj_mx_drive = availability_matrix(g.graph, points)
adj_mx_inter = availability_matrix(inter, points,graph_type=[GraphType.PUBLIC_TRANSPORT, GraphType.WALK])

In [ ]:
grader = AdvancedGrader(local_crs)
cri = grader.get_criteria(graded_terr=graded_terr,points=points,polygons=settlements,r_stops=r_stops,b_stops=b_stops,aero=aero,adj_mx_drive=adj_mx_drive,adj_mx_inter=adj_mx_inter)
cri

## indicators terr

In [ ]:
# inter = nx.read_graphml('/Users/polina/Desktop/github/transport_frames/preprocessed/Тульская_область/Тульская_область_inter.graphml')
# inter = he.prepare_graph(inter)
poly = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/settlement.geojson').reset_index()
settlements = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/settlement.geojson')

b_stops = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/bus_stop.geojson')
r_stops = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/railway_station.geojson')
aero = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/aerodrome.geojson')
points = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Тульская_область_region_points.geojson')
regions_gdf = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/district.geojson')

In [ ]:
custom_territory=poly.iloc[[5]]
custom_territory_center = custom_territory.geometry.representative_point()
custom_territory_center = gpd.GeoDataFrame([{'geometry': custom_territory_center.iloc[0]}], crs=custom_territory_center.crs).to_crs(32637)


In [ ]:
PLACEHOLDER = gpd.GeoDataFrame(geometry=[]) # если нет данных о сервисе

railway_stations = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/railway_station.geojson')
railway_roads = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/railway.geojson')
bus_stops = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/bus_stop.geojson')
bus_routes = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/preprocessed/Тульская_область/Тульская_область_bus_routes.geojson')
fuel_stations = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Копия fuel.geojson')
ferry_terminal = PLACEHOLDER
local_aerodrome = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/aerodrome.geojson')
local_aerodrome['geometry'] = shapely.centroid(local_aerodrome['geometry']).set_crs(local_aerodrome.crs)
# international_aerodrome = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_international_aerodrome.geojson')
# либо:
international_aerodrome = local_aerodrome[local_aerodrome['aerodrome:type']=='international']
international_aerodrome['geometry'] = shapely.centroid(international_aerodrome['geometry']).set_crs(international_aerodrome.crs)

# Природные объекты
water = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Копия water.geojson')
nature_reserve = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Копия nature_reserve.geojson')


In [ ]:
services = {
    'railway_stops' : railway_stations,
    'fuel' : fuel_stations,
    'ports' : ferry_terminal,
    'local_aero' : local_aerodrome,
    'international_aero' : international_aerodrome,
    'oopt' : nature_reserve,
    'water_objects' : water,
    'train_paths' : railway_roads,
    'bus_stops' : bus_stops,
    'capital': ox.geocode_to_gdf('N34389350',by_osmid=True),
    'points': points,
    'bus_routes' : bus_routes,
    'region_centers': cities,
    'neud_center': custom_territory_center

}

In [ ]:
G = assign_services_names_to_nodes(services,momepy.nx_to_gdf(g.graph)[0],g.graph,crs = 32637)

In [ ]:
t = indicator_territory(G,poly.iloc[[5]],3857,regions_gdf, points, cities, services, *momepy.nx_to_gdf(g.graph))
t

In [ ]:
inter = he.prepare_graph(inter)

In [ ]:
b = indicator_area(G, inter, services, [settlements])

In [ ]:
b[0]